# Deploy and destroy the Lambda infrastructure

This notebooks contains scripts that help build and deploy a binary using the reader library to AWS Lambda.

In [5]:
import os

region_name="us-east-2"
binary_name="lambda"
aws_profile=os.environ["AWS_PROFILE"] # Specify the profile you want to use from your .aws/credentials file with the AWS_PROFILE env variable

Create a terraform container containing the binary to be deployed and the infra script, with `terraform init` already executed.

In [3]:
%%bash -s "$binary_name"
date
DOCKER_BUILDKIT=1 docker build -q \
    -f Dockerfile \
    --build-arg BIN_NAME=$1 \
    --target terraform \
    -t cloudfuse/cloud-reader-terraform \
    ..

Mon Aug  9 09:01:34 CEST 2021
sha256:f2865cf57273edf9abb40b9563f3c8cde8dd064d631d82090426a868478d23d3


In [4]:
%%bash -s "$aws_profile" "$region_name" "$binary_name"
date
docker run \
    --rm \
    -v $HOME/.aws/credentials:/creds:ro \
    -v cloud-reader-tf:/mnt/state_vol \
    cloudfuse/cloud-reader-terraform apply -var="profile=$1" -var="region_name=$2" -var="binary_name=$3" -auto-approve

Mon Aug  9 09:01:44 CEST 2021

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create
 <= read (data resources)

Terraform will perform the following actions:

  # module.lambda_function.data.aws_iam_policy_document.logs[0] will be read during apply
  # (config refers to values not yet known)
 <= data "aws_iam_policy_document" "logs"  {
      + id   = (known after apply)
      + json = (known after apply)

      + statement {
          + actions   = [
              + "logs:CreateLogGroup",
              + "logs:CreateLogStream",
              + "logs:PutLogEvents",
            ]
          + effect    = "Allow"
          + resources = [
              + (known after apply),
              + (known after apply),
            ]
        }
    }

  # module.lambda_function.aws_cloudwatch_log_group.lambda[0] will be created
  + resource "aws_cloudwatch_log_group" "lambda" {
      + arn               =

# Destroy AWS Lambda function

In [7]:
%%bash -s "$aws_profile" "$region_name" "$binary_name"
date
docker run \
    --rm \
    -v $HOME/.aws/credentials:/creds:ro \
    -v cloud-reader-tf:/mnt/state_vol \
    cloudfuse/cloud-reader-terraform destroy -var="profile=$1" -var="region_name=$2" -var="binary_name=$3" -auto-approve

Fri Aug  6 11:08:21 CEST 2021
module.lambda_function.aws_cloudwatch_log_group.lambda[0]: Refreshing state... [id=/aws/lambda/cloud-reader-benchmark]
module.lambda_function.aws_iam_role.lambda[0]: Refreshing state... [id=cloud-reader-benchmark]
module.lambda_function.aws_lambda_function.this[0]: Refreshing state... [id=cloud-reader-benchmark]
module.lambda_function.aws_iam_policy.logs[0]: Refreshing state... [id=arn:aws:iam::615900053518:policy/cloud-reader-benchmark-logs]
module.lambda_function.aws_iam_policy_attachment.logs[0]: Refreshing state... [id=cloud-reader-benchmark-logs]

Note: Objects have changed outside of Terraform

Terraform detected the following changes made outside of Terraform since the
last "terraform apply":

  # module.lambda_function.aws_iam_role.lambda[0] has been changed
  ~ resource "aws_iam_role" "lambda" {
        id                    = "cloud-reader-benchmark"
      ~ managed_policy_arns   = [
          + "arn:aws:iam::aws:policy/AmazonS3FullAccess",
     

In [8]:
%%bash
# WARNING: deleting the volume before running `terraform destroy` will make it impossible to tear down the infrastructure automatically
date
docker volume rm cloud-reader-tf

Fri Aug  6 11:08:53 CEST 2021
cloud-reader-tf


In [9]:
%%bash
date
docker image rm cloudfuse/cloud-reader-terraform

Fri Aug  6 11:08:53 CEST 2021
Untagged: cloudfuse/cloud-reader-terraform:latest
Deleted: sha256:eb6f623bf02e87581f90b3be10ab94fa50e4dc77bc5fda97b5e8ca3732978b54
